In [2]:
import pandas as pd
import stanza
import spacy
from transformers import BertTokenizer, BertForSequenceClassification
import torch

2023-09-28 11:53:01.372549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Stanza kullanarak Türkçe dil modeli yüklüyoruz 
stanza.download("tr")
nlp_stanza = stanza.Pipeline("tr")

2023-09-28 11:53:09 INFO: Downloading default packages for language: tr (Turkish) ...
2023-09-28 11:53:10 INFO: File exists: /home/abdullah/stanza_resources/tr/default.zip
2023-09-28 11:53:14 INFO: Finished downloading models and saved to /home/abdullah/stanza_resources.
2023-09-28 11:53:14 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-09-28 11:53:14 INFO: Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |
| depparse  | imst_charlm   |
| ner       | starlang      |

2023-09-28 11:53:14 INFO: Using device: cpu
2023-09-28 11:53:14 INFO: Loading: tokenize
2023-09-28 11:53:14 INFO: Loading: mwt
2023-09-28 11:53:14 INFO: Loading: pos
2023-09-28 11:53:15 INFO: Loadin

In [14]:

# Verileri bir liste olarak okutalım
texts = []
aspects = []
sentiments = []

train_file_path = "restaurants_train_turkish.txt"

# Dosyaları okuyup verileri ayrıştırmak
def read_train_data(file_path):
    with open(file_path, "r") as train_file:
        train_lines = train_file.readlines()
    for i in range(0, len(train_lines), 3):
        texts.append(train_lines[i].strip())
        aspects.append(train_lines[i + 1].strip())
        sentiments.append(train_lines[i + 2].strip())


read_train_data(train_file_path)

print("Train Texts:", texts)
print("Train Aspects:", aspects)
print("Train Sentiments:", sentiments)
print('-'*100)


Train Texts: ['$T$ sahane evet ama servis rezalet.', 'Manzara sahane evet ama $T$ rezalet.', 'Soguk su isteyince, soguk yok, butun sulari disari cikardik diyen garson, siparisten tam 2 saat sonra gelen yemekler, yemegin yaninda soylenen roka ve pureden eser olmamasi, istedigimiz 2 $T$ de kalmamis olmasi.', 'Soguk su isteyince, soguk yok, butun sulari disari cikardik diyen $T$, siparisten tam 2 saat sonra gelen yemekler, yemegin yaninda soylenen roka ve pureden eser olmamasi, istedigimiz 2 mezenin de kalmamis olmasi.', '$T$ iyi hos, lezzetler iyi ama heyecan verici bi taraflari yok, iyi bir baligi iyi bir sekilde izgara yapmak artik atla deve bi olay degil.', 'Yemekler iyi hos, $T$ iyi ama heyecan verici bi taraflari yok, iyi bir baligi iyi bir sekilde izgara yapmak artik atla deve bi olay degil.', 'bu servise bu $T$ ise cok faZla.', 'Lacivert beni $T$ dusukluguyle cok sasirtti', 'Buraya bu kadar iyi yorum yapanlar her halde daha önce hiç $T$ yemediler.', 'Kalamardan ve koladan önce $T$

In [22]:
# Veri çerçevesini oluşturalım
df = pd.DataFrame({'Tweet': texts, 'Aspect': aspects, 'Sentiment': sentiments})

In [23]:
df

,Tweet,Aspect,Sentiment
0,$T$ sahane evet ama servis rezalet.,Manzara,Positive
1,Manzara sahane evet ama $T$ rezalet.,servis,Negative
2,"Soguk su isteyince, soguk yok, butun sulari di...",mezenin,Negative
3,"Soguk su isteyince, soguk yok, butun sulari di...",garson,Negative
4,"$T$ iyi hos, lezzetler iyi ama heyecan verici ...",Yemekler,Positive
...,...,...,...
1380,Tereyağlı pilavları ve $T$ da çok başarılı.,tava yoğurdu,Positive
1381,Üzerine bal ve fındık dökülmüş $T$ çok cezbedi...,sütlaçları,Positive
1382,Üzerine bal ve fındık dökülmüş sütlaçları çok ...,sütlaç tadını,Negative
1383,Ayrıca hergun çıkardıkları farklı $T$ de çok b...,zeytinyağlı yemekleri,Positive


In [24]:
# Aspect tanıma işlemi için stanza kullaniyoruz
def extract_aspects(text):
    doc = nlp_stanza(text)
    aspects = [ent.text for sent in doc.sentences for ent in sent.ents]
    return aspects

In [48]:
df['Aspects'] = df['Tweet'].apply(extract_aspects)

In [49]:
# Duygu analizi için bir önceden eğitilmiş türkçe modeli
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=3)  # 3 sınıf için (Positive, Negative, Neutral)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
def tokenize_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    return inputs

In [51]:
df['Tokenized'] = df['Tweet'].apply(tokenize_text)

In [52]:
def sentiment_analysis(tokenized_text):
    input_ids = tokenized_text['input_ids']
    attention_mask = tokenized_text['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

    return predicted_labels.item()

In [53]:
df['Predicted_Sentiment'] = df['Tokenized'].apply(sentiment_analysis)

In [54]:
# Aspect bazlı duygu tahminleri
aspect_sentiments = {}
for index, row in df.iterrows():
    tweet = row['Tweet']
    aspects = row['Aspects']
    sentiment = row['Predicted_Sentiment']
    
    for aspect in aspects:
        if aspect not in aspect_sentiments:
            aspect_sentiments[aspect] = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
        
        if sentiment == 0:
            aspect_sentiments[aspect]['Negative'] += 1
        elif sentiment == 1:
            aspect_sentiments[aspect]['Neutral'] += 1
        elif sentiment == 2:
            aspect_sentiments[aspect]['Positive'] += 1

In [55]:
aspect_sentiments

{'$T$': {'Positive': 171, 'Negative': 143, 'Neutral': 611},
 '2 $T$': {'Positive': 0, 'Negative': 1, 'Neutral': 0},
 'Kalamardan': {'Positive': 0, 'Negative': 1, 'Neutral': 0},
 '6 tl': {'Positive': 0, 'Negative': 1, 'Neutral': 0},
 '$T$👌': {'Positive': 1, 'Negative': 0, 'Neutral': 0},
 "Hard Rock Cafe'de": {'Positive': 1, 'Negative': 0, 'Neutral': 1},
 'Avrupa': {'Positive': 1, 'Negative': 0, 'Neutral': 1},
 "Amerika'dakilerle": {'Positive': 1, 'Negative': 0, 'Neutral': 1},
 'Bosken': {'Positive': 0, 'Negative': 0, 'Neutral': 1},
 "Beşiktaş'ta": {'Positive': 1, 'Negative': 0, 'Neutral': 1},
 'Vanilla': {'Positive': 0, 'Negative': 1, 'Neutral': 0},
 'Pazar günü': {'Positive': 0, 'Negative': 0, 'Neutral': 1},
 'Big Chefs': {'Positive': 1, 'Negative': 0, 'Neutral': 1},
 'Wrapten': {'Positive': 0, 'Negative': 0, 'Neutral': 1},
 "Tuzla'da": {'Positive': 0, 'Negative': 1, 'Neutral': 1},
 'Marmara': {'Positive': 1, 'Negative': 0, 'Neutral': 0},
 "İstanbul'da": {'Positive': 1, 'Negative': 1, 